In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from imutils import paths
import cv2 as cv

from tensorflow.keras.applications import MobileNetV2 
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model

from imutils.video import VideoStream
import imutils

In [2]:
dataset = '/Users/kunal/Desktop/Age-Detection/dataset'
imagePaths = list(paths.list_images(dataset))

In [ ]:
imagePaths

In [3]:
data = []
labels = []

for i in imagePaths:
    label = i.split(os.path.sep)[-2]
    labels.append(label)
    image = load_img(i, target_size = (224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)

In [ ]:
data[0]

In [ ]:
labels[0]

In [4]:
data = np.array(data, dtype = 'float32')
labels = np.array(labels)

In [5]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [6]:
train_X, test_X, train_Y, test_Y = train_test_split(data, labels, test_size = 0.20, random_state = 10, stratify = labels)

In [7]:
baseModel = MobileNetV2(weights='imagenet', include_top=False, input_tensor= Input(shape = (224,224,3)))

In [ ]:
baseModel.summary()

In [8]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size = (5,5))(headModel)
headModel = Flatten(name = 'Flatten')(headModel)
headModel = Dense(128, activation = 'relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation = 'softmax')(headModel)

model = Model(inputs = baseModel.input, outputs = headModel)

In [9]:
for layer in baseModel.layers:
    layer.trainable = False

In [ ]:
model.summary()

In [10]:
learning_rate = 0.01
Epochs = 10
batch_size = 12

opt = Adam(lr=learning_rate, decay = learning_rate/Epochs)

model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy'])

H = model.fit(
    train_X, train_Y, batch_size = batch_size,
    steps_per_epoch=len(train_X)//batch_size,
    validation_data=(test_X, test_Y),
    validation_steps=len(test_X)//batch_size,
    epochs = Epochs
)


Epoch 1/10
882/882 [==============================] - 248s 278ms/step - loss: 0.4764 - accuracy: 0.8634 - val_loss: 0.2119 - val_accuracy: 0.9178
Epoch 2/10
882/882 [==============================] - 254s 288ms/step - loss: 0.2540 - accuracy: 0.8995 - val_loss: 0.1924 - val_accuracy: 0.9273
Epoch 3/10
882/882 [==============================] - 269s 305ms/step - loss: 0.2262 - accuracy: 0.9062 - val_loss: 0.1991 - val_accuracy: 0.9227
Epoch 4/10
882/882 [==============================] - 276s 313ms/step - loss: 0.2125 - accuracy: 0.9190 - val_loss: 0.1914 - val_accuracy: 0.9288
Epoch 5/10
882/882 [==============================] - 279s 317ms/step - loss: 0.1976 - accuracy: 0.9237 - val_loss: 0.1897 - val_accuracy: 0.9250
Epoch 6/10
882/882 [==============================] - 284s 322ms/step - loss: 0.1987 - accuracy: 0.9228 - val_loss: 0.1899 - val_accuracy: 0.9295
Epoch 7/10
882/882 [==============================] - 286s 325ms/step - loss: 0.1770 - accuracy: 0.9281 - val_loss: 0.1831 -

In [11]:
model.save('model3.model')

INFO:tensorflow:Assets written to: model3.model/assets
